In [ ]:
#importing packages

from SCHyA import SCHyA as hy
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import scipy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm

import byotrack

In [ ]:
# Replacement for extract fluorescence in schya

def extract_intensities(video, tracks, patch_size):
    """Extract intensities at the track location from the given video"""
    intensities = np.zeros((len(tracks), len(video)))
    for frame_id, frame in enumerate(tqdm.tqdm(video)):
        for track_id, track in enumerate(tracks):
            point = track[frame_id]

            i = int(point[0] - patch_size / 2)
            j = int(point[1] - patch_size / 2)

            intensities[track_id, frame_id] = frame[max(0, i) : i + patch_size, max(0, j) : j + patch_size].mean()
    return intensities

In [ ]:
# Unused
#input path to your .csv from ICY
csv_path = '/home/noah/Documents/NoahT2022/Data/SingleCell_Tracking/ALI_comp/redTracks.csv'

positions = byotrack.Track.tensorize(byotrack.Track.load("tracks.pth")).numpy()
positions_reshaped = np.transpose(positions, (1,0,2))

In [ ]:
# Load videos
#gcamp_path = "../pasteur/data/long movies_tracks/6X_green_1000_2bin.avi"
#tdtomato_path = "../pasteur/data/long movies_tracks/6X_red_1000_2bin.avi"
gcamp_path = '/home/noah/Desktop/cellsegtest/segTestNew/shortStack_adjusted'
tdtomato_path ='/home/noah/Desktop/cellsegtest/segTestNew/shortStack_adjusted'

gcamp_video = byotrack.Video(gcamp_path)[:100]
tdtomato_video = byotrack.Video(tdtomato_path)[:100]

# # To be deleted, we should keep clearer names
# vid = gcamp_video
# red_vid = tdtomato_video

In [ ]:
# Reload tracks

tracks = byotrack.Track.load("tracks.pt")positions, vid, red_vid = hy.Read_Data_TIFseq_byo(numpy_path, green_channel_path, red_channel_path)

In [ ]:
# Extract calcium and control intensities

intensities_raw = extract_intensities(vid, tracks, 9)

intensities_red = extract_intensities(red_vid, tracks, 9)

In [ ]:
# 709 is an interesting tracks with 2/3 firings

plt.plot(intensities_raw[709])

In [ ]:
plt.plot(intensities_red[709])

In [ ]:
# Unused
positions, vid, red_vid = hy.Read_Data_TIFseq_byo(numpy_path, vid_path, red_vid_path)
positions_reshaped = np.transpose(positions, (1,0,2)) #https://stackoverflow.com/questions/68700008/difference-between-just-reshaping-and-reshaping-and-getting-transpose

In [ ]:
# Should be useless now

def npy_remove_incomplete_tracks(tracks_reshaped):

    complete_tracks = []
    incomplete_tracks = []
    indexes = []

    for i, track in enumerate(tracks_reshaped):
        hasNaN = np.isnan(track).any()
        if hasNaN:
            print('nan')
            incomplete_tracks.append(track)
            indexes.append(i)
        else:
            complete_tracks.append(track)
    
    return np.asarray(complete_tracks), [incomplete_tracks, indexes]

posit_corrected, removed_tracks = npy_remove_incomplete_tracks(positions_reshaped)

In [ ]:
# Unused

#Extract the fluorescence intensity in the GCaMP channel throughout the video for each neuron

#size of ROI
# dim = 9
dim=9
# subROI_Circle_size = 4
subROI_Circle_size = 3
LookBack = 1 #resistance of tracking to movement i.e. how quickly a tracked neuron can move before position resets

#Extract large ROI and remove points too close to edge
intensities_raw, posit_corrected_raw = hy.Extract_Fluorescence(positions_reshaped, vid, dimention = dim)

#extract signal from subROI around neuron
# intensities1, posit_corrected1, neuron_pts = hy.SingleCellIntensities(vid, posit_corrected_raw, dim, subROI_Circle_size, LookBack)

In [ ]:
#TODO using the sequence loader gives frames in wrong format for cv2 rgb2gray function - need to run conversion either in data loader or in single cell function (or both)

In [ ]:
#Figure1 subplots 
import matplotlib as mpl
keyCells = range(30)
keyCells = [4,8,12,29]

mpl.rcParams.update({'font.size': 18, "text.usetex": True})
fig, axs = plt.subplots(2, len(keyCells), sharex =True, sharey=True, figsize=(30, 5))


# axs.xlabel("x", fontsize=80)
# axs.ylabel("y", fontsize=80)
# axs.xlim(-3, 3)
# axs.ylim(-1.05, 1.05)

styles = ["o", "^", "s", "X", "*"]
default_color_iter = plt.rcParams["axes.prop_cycle"]()




axs[0,0].set_ylabel('Raw Signals')
axs[1,0].set_ylabel('Sub ROI Signals')

for i, neuron in enumerate(keyCells):
    axs[0,i].set_title('Neuron: ' + str(neuron))

    axs[0,i].plot(intensities_raw[neuron])
    axs[0,i].grid()
    axs[1,i].grid()
    axs[1,i].plot(intensities_raw[neuron]) #these might not line up if there is an issue with points being removed!
    
# fig.text(0.5, -0.05, 'Frames', ha='center', fontsize = 30)
# fig.text(0.04, 0.5, 'common Y', va='center', rotation='vertical')
# plt.savefig("/home/noah/Documents/NoahT2022/utopiaFigures/rawvsROI_signals.svg", format = "svg")
plt.show()

In [ ]:
CaSignal = hy.ICAdecorr(intensities_raw, intensities_red, 0.5, 10)

#Plot Signal
for i in range(len(CaSignal)):
    plt.plot(CaSignal[i])
plt.title('Extracted Calcium Signal Plot')
plt.xlabel('Frame')
plt.ylabel('Ca Intensity')
plt.show()

In [ ]:
plt.plot(CaSignal[709])

In [ ]:
hy.plot_heatmap(CaSignal, 'Heatmap of Extracted Calcium Signal', 'Ca Signal Intensity')

In [ ]:
#Set Polynomial Degree
poly_deg = 1

detrended = hy.detrend_all(intensities_raw.tolist(), poly_deg)

In [ ]:
plt.plot(detrended[709])

In [ ]:
alpha = 1e-4 #Tune threshold for coherence to gaussian distribution - need to use detrended data with this function
noise = []
firing_neurons = []

for intensities in detrended:
    a, b = scipy.stats.normaltest(intensities)
    if b >= alpha:
        noise.append(intensities)
    else:
        firing_neurons.append(intensities)
        
print(len(noise), len(firing_neurons))

In [ ]:
# Unused

alpha = 1e-5 #Tune threshold for coherence to gaussian distribution - need to use detrended data with this function
detrended2, posit_corrected2, removed = hy.Gaussian_noise_filter(detrended, alpha, posit_corrected)

In [ ]:
fig, axs = plt.subplots(4, 4, sharex = 'col', sharey='row', figsize=(20, 20))
for i, cell in enumerate(firing_neurons[0:16]):
    if i<4:
        axs[0,i].plot(cell)
    elif i >=4 and i < 8:
        axs[1,i-4].plot(cell)
    elif i >=8 and i < 12:
        axs[2,i-8].plot(cell)
    elif i >=12 and i < 16:
        axs[3,i-12].plot(cell)

plt.show()

In [ ]:
#Create Raster Plot using CAIMAN's FOOPSI function - Denoising and Deconvolution

#foopsi
Foopsi_ca, spikes_signal_dR = hy.FOOPSI_all(firing_neurons)

#Extract Raster Plot Data

#Threshold (could use a theoretical value for threshold! - See CAIMAN Docs - but trial & error is also fine)
#USE FOOPSI EVALUATION CELL TO TUNE THIS PARAMETER (0.04 works well)
spike_thresh_dR = 0.4

raster_array_dR = hy.Find_Raster_adaptive2(spikes_signal_dR, spike_thresh_dR)

In [ ]:
#Display Results

#plot raster
#zero values from way array was made array give large syncronous train at start of the signal - should fix (.append method?)
plt.figure(2)
plt.eventplot(raster_array_dR,linelengths = 0.6)
plt.ylabel('Neuron')
plt.xlabel('Frame')
plt.title('Raster Plot of Neural Activity of Hydra')
plt.xlim((1,len(raster_array_dR[0])))
plt.show()

In [ ]:
np.isnan(tracks[:,2]).any()

In [ ]:
def hasNaNs(index):
    i = index
    hasNaN = np.isnan(tracks[:,i]).any()
    return hasNaN

def hasNaNs2(column):
    # i = index
    hasNaN = np.isnan(column).any()
    return hasNaN

# any(np.isnan(tracks[:,2]))

In [ ]:
complete_tracks = []
incomplete_tracks = []
indexes = []
for i in range(tracks.shape[1]):
    hasNaN = np.isnan(tracks[:,i]).any()
    if hasNaN:
        incomplete_tracks.append(tracks[:,i])
        indexes.append(i)
    else:
        complete_tracks.append(tracks[:,i])

In [ ]:
print(indexes)
print('number Removed: ', len(indexes))
print('num kept: ', len(complete_tracks))

In [ ]:
completetracks = [hasNaNs2(track[:,i]) for  ]

In [ ]:
print(tracks.shape)

df = pd.DataFrame(data = tracks.tolist())

In [ ]:
df

In [ ]:
df.apply(hasNaNs2, axis=0)

In [ ]:
df.columns[df.isna().any()].tolist()

In [ ]:
num_frames = tracks.shape[1]

In [ ]:
#reshape tracks
posit_corrected = hy.remove_incomplete_tracks(tracks, num_frames)

In [ ]:
len(posit_corrected)

In [ ]:
positions = (pd.read_csv(csv_path,usecols=['TrackID','t','x','y']))

trackIDs = positions.TrackID.unique()
grouped = positions.groupby(['TrackID'])

posit = np.asanyarray([grouped.get_group(neuron).loc[:, ['x', 'y']].values for neuron in trackIDs], dtype='object')

In [ ]:
posit[0]

In [ ]:
positions = pd.read_csv(csv_path, dtype = 'object', converters={'0': pd.eval})
# positions.drop(positions.columns[0], axis=1, inplace=True)

positions

In [ ]:
from ast import literal_eval


In [ ]:
posit_corrected = complete_tracks

In [ ]:
len(posit_corrected)

In [ ]:
_, red, green = hy.Read_Data_TIFseq(vid_path, red_vid_path, csv_path)